In [53]:
import glob

from bs4 import BeautifulSoup
import esanpy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

/Users/wararaki/.pyenv/versions/3.6.3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/wararaki/.pyenv/versions/3.6.3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
esanpy.stop_server()
esanpy.start_server()

ERROR:esanpy:Failed to stop Elasticsearch process: [Errno 3] No such process


In [3]:
natsume_files = glob.glob('data/natsume/files/*.html')
dazai_files = glob.glob('data/dazai/files/*.html')

In [4]:
def get_soup(filepath):
    with open(filepath, encoding='shift_jis') as f:
        soup = BeautifulSoup(f, 'html.parser', from_encoding='shift_jis')
    return soup

def get_texts(filepaths):
    texts = [get_soup(filepath).body.text for filepath in tqdm(filepaths)]
    return texts

In [5]:
natsume_texts = get_texts(natsume_files)
dazai_texts = get_texts(dazai_files)

100%|██████████| 276/276 [00:06<00:00, 43.05it/s]


In [6]:
def custom_tokenizer(text):
    return esanpy.analyzer(text, analyzer='kuromoji_neologd')

In [78]:
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=10000, min_df=2)
vectorizer.fit(natsume_texts+dazai_texts)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=2,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function custom_tokenizer at 0x11da9e620>, use_idf=True,
        vocabulary=None)

In [79]:
x = vectorizer.transform(natsume_texts + dazai_texts)

y = [i for i, _ in enumerate(natsume_texts)]
y.extend([i for i, _ in enumerate(dazai_texts)])

In [80]:
lr_l2 = LogisticRegression(penalty='l2', class_weight='balanced', max_iter=1000)

In [81]:
lr_l2.fit(x, y)

/Users/wararaki/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/wararaki/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=1000,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [82]:
features = []
for coef, feature in zip(lr_l1.coef_.flatten(), vectorizer.get_feature_names()):
    if coef > 0.0:
        features.append({'coef': coef, 'word': feature})
df = pd.DataFrame(features).sort_values(by=['coef'], ascending=False)
display(df.shape)
df.head(10)

(448, 2)

,coef,word
125,0.567561,世帯
100,0.562684,みょう
164,0.506787,判
74,0.427111,づち
440,0.417119,顧慮
32,0.383494,く
45,0.344142,ころ
425,0.343578,遊戯
401,0.343309,襖
91,0.341097,ひばり


In [83]:
features = []
for coef, feature in zip(lr_l1.coef_.flatten(), vectorizer.get_feature_names()):
    if coef < 0.0:
        features.append({'coef': coef, 'word': feature})
df = pd.DataFrame(features).sort_values(by=['coef'], ascending=True)
display(df.shape)
df.head(10)

(9552, 2)

,coef,word
2069,-0.978810,みき
919,-0.955016,ごめんなさい
539,-0.947735,お話
3029,-0.926429,仕合せ
5884,-0.919000,敷島
4942,-0.906620,市場
7599,-0.905280,箱根
2261,-0.902537,ゆふ
8363,-0.900803,親
3409,-0.894429,六尺
